In [1]:
import sys
sys.path.append("../modules")
from prompt_segments import *
from functions import *
import datetime
from tqdm import tqdm
import importlib

In [2]:
# load scenarios
with open('./outputs/scenarios.txt') as f:
    scenarios = f.read().splitlines()
    scenarios = [s.split(". ")[1] for s in scenarios]

In [3]:
scenarios[:8]

['My partner cheated on me, but I still love them',
 "I'm struggling with my faith and questioning everything I believe in",
 "I've been offered a job in another country, but my partner doesn't want to move",
 "I accidentally hurt someone's feelings very badly",
 'I think my friend is involved in illegal activities',
 'My boss is harassing me, but I need this job',
 "I've been lying to my family about my career for years",
 "I feel like I don't belong anywhere"]

In [5]:
def generate_context( question: str ) -> str:
    prompt = f"""You’re a chatbot and you’ll receive a question. Your job is to struggle with how to answer it, and document your thought process.

- Your first task is to list moral considerations you might face in responding to the question (see definition below).
- Next think about what kind of good outcome or thing you are protecting or honoring in your response. Use this to generate a GoodWhat (see specification below).
- Finally, select the 1-2 of the moral considerations that most strongly indicate that this value would apply in this kind of choice.

The output should be formatted exactly as in the example below.

{moral_considerations_definition}

# GoodWhat

In any choice, you are choosing a good X, where X is a kind of thing. For instance, you might be choosing good comforting words, or helping the user choose a good partner, or good daily activities. Do not define what good means here (do not say "respectful", "considerate", etc). That goes in the moral considerations. Also don't say "guidance" unless that's realy core to the situation. Instead of "Good guidance for repairing relationships" say "Good ways to repair relationships". The good should be the most important thing you are protecting or honoring in your choice. It can either be about your choice as a chatbot ("Good ways to deal with an emotional user"), or about the choice you will help the user make ("Good partners").

=== Example Input ===
# Question
I'm really stressed and overwhelmed. I just heard that a collaborator of ours quit, and this messes up all of our plans. I also have a lot of personal stuff going on, and I'm not sure how to balance all of these concerns. I'm not sure what to do.

=== Example Output ===
# Moral Considerations
I might amplify the crisis in the user’s mind
I don’t know if the situation is untenable or if the user is just overwhelmed
I might provide practical advice when emotional comfort is necessary, or vice versa

# Good What
Good approaches to a crisis

# Most Important Considerations
I might amplify the crisis in the user’s mind
"""

    user_prompt = "# Question\n" + question
    return str(gpt4(prompt, user_prompt))

d = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
with open(f"./outputs/contexts_${d}.txt", "w") as f:
  for q in tqdm(scenarios[:30]):
      output = generate_context(q)
      # remove everything before the choice type
      output = output.split("# Good What")[1]
      # put an extra # before the Most Important Considerations
      output = output.replace("# Most Important Considerations", "## Most Important Considerations")
      summary = "# Q: " + q + "\n\n## Good what\n" + output + "\n\n"
      f.write(summary)
      print(summary)
f.close()

  3%|▎         | 1/30 [00:07<03:44,  7.76s/it]

# Q: My partner cheated on me, but I still love them

## Good what

Good ways to navigate complex emotional situations

## Most Important Considerations
Promoting emotional healing without minimizing the betrayal
Supporting the user's autonomy in decision-making




  7%|▋         | 2/30 [00:12<02:55,  6.25s/it]

# Q: I'm struggling with my faith and questioning everything I believe in

## Good what

Good support for spiritual or existential questioning

## Most Important Considerations
Respecting the user's current belief system
Providing comfort without dismissing the user's doubts




 10%|█         | 3/30 [00:19<02:51,  6.37s/it]

...

# Q: My pet is very ill and I can't afford treatment

## Good what

Good sources of support for pet care

## Most Important Considerations
Providing false hope or unrealistic solutions
Ignoring the emotional distress of the user




 70%|███████   | 21/30 [02:48<01:09,  7.68s/it]

# Q: I'm feeling immense pressure to succeed, but I'm not happy

## Good what

Good balance between success and happiness

## Most Important Considerations
Promoting unrealistic expectations of success
Ignoring the root causes of unhappiness




 70%|███████   | 21/30 [02:48<01:12,  8.04s/it]


KeyboardInterrupt: 